In [1]:
#cargar datos y calculos mágicos
import numpy as np
import pandas as pd

#plots 
from matplotlib import pyplot as plt 
from plotly import graph_objects as go
from plotly import express as px
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot 
import seaborn as sns
import time

#tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM,GRU
# from tensorflow.keras.models import Sequential

#escalar datos
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import joblib #guardar el scaler






tf.random.set_seed(0)

In [2]:
np80_scaler = joblib.load('modelos/np80_scaler.gz')
# model=tf.keras.models.load_model('modelos/GRU80_20_y_06-12-2024.keras')

In [3]:
df = pd.read_parquet('datos15til24resampledwxwy.parquet.gzip')
# df=df.loc['2019-05':].copy()

In [4]:


df.columns = df.columns.str.lower()


df['ds']=df.index
df['ds']


#cambiar nombres
df = df.rename(columns={
'temp ext':'y',
'hum ext':'humext',
'temp int':'tempint',
'humedad int':'humint',
'presión' : 'pres',
'temp ap': 'tempap',
"rad solar t": "radsolar",
})


#borro las columnas que siempre son 0

siemprecero=['uv','solar rad','evap','evap anual','horas de sol','rg-11 rain']
df=df.drop(columns=siemprecero,axis=1)

#borro las filas que tienen más de un valor para el mismo horario (no sé por qué pasó esto pero pasó)

# df=df.drop_duplicates(subset=df['ds'])

df=df.fillna(method='ffill')


df.head()

C:\Users\soysa\AppData\Local\Temp\ipykernel_7972\462546089.py:29: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,y,humext,pr,rafaga 10m,tasa precip,lluvia hoy,pres,cont lluvia,tempint,humint,viento actual,wind chill,heat index,tempap,radsolar,dir actual,lluvia desde 00:00,wx,wy,ds
ds,,,,,,,,,,,,,,,,,,,,
2015-03-31 13:00:00,26.5,31.0,8.0,20.9,0.0,0.0,1014.5,3.3,26.6,48.0,15.245000,26.5,26.5,24.4,462.0,211.500000,3.3,-8.863656,-7.049562,2015-03-31 13:00:00
2015-03-31 14:00:00,27.3,26.0,6.1,23.4,0.0,0.0,1013.6,3.3,26.6,45.0,15.133333,27.3,26.5,24.0,175.0,193.483333,3.3,-10.551653,-3.243121,2015-03-31 14:00:00
2015-03-31 15:00:00,28.2,26.0,6.9,22.0,0.0,0.0,1012.9,3.3,25.8,45.0,16.156667,28.2,27.1,25.6,0.0,188.616667,3.3,-11.587035,-2.221017,2015-03-31 15:00:00
2015-03-31 16:00:00,28.6,30.0,9.4,14.8,0.0,0.0,1012.0,3.3,25.8,45.0,11.350000,28.6,27.5,25.9,0.0,193.116667,3.3,-8.023175,-1.828481,2015-03-31 16:00:00
2015-03-31 17:00:00,28.6,31.0,9.8,14.8,0.0,0.0,1011.9,3.3,25.8,42.0,11.456667,28.6,27.6,26.7,0.0,174.400000,3.3,-8.238028,0.288265,2015-03-31 17:00:00


In [5]:
timestamp_s = df['ds'].map(pd.Timestamp.timestamp)
timestamp_s=timestamp_s-timestamp_s.iloc[0]

day = 24*60*60 #esto porque los datos estÃ¡n en minutos como minima resoluciÃ³n
year = (365.2425)*day

df['day_sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['day_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['year_sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['year_cos'] = np.cos(timestamp_s * (2 * np.pi / year))

In [6]:
columnas_a_usar=['y','wx','wy','humext','pr',
                 'day_cos','day_sin','year_cos','year_sin']



df_input=df[columnas_a_usar]

df_input.shape

(84336, 9)

In [7]:
#constantes de corte
len_100=len(df_input)
len_70=int(len_100*.7)
len_80=int(len_100*.8)
len_90=int(len_100*.9)

#constantes de predicción

n_lookback=24*7
n_forecast=24*2
n_columnas=9

In [8]:
def array_to_X_y(array_datos,n_lookback=24*7,n_forecast=48):
    # df_np= df.to_numpy()
    #print(np.shape(df_np))
    X=[]
    y=[]
    for i in range(len(array_datos)-n_lookback-n_forecast): #itera X - 168 - 48 por defecto
        row = [r for r in array_datos[i:i+n_lookback]]      #hace un array de forma de (imax, n_lookback , columnas de input )
        X.append(row)
        label=[array_datos[i+n_lookback:i+n_lookback+n_forecast,0]] #esto cambiar por nombre de columna porque si no va a ser horrible 
                                                                    # 0 es temperatura
        # label=[df_np[i+n_lookback:i+n_lookback+n_forecast,0],df_np[i+n_lookback:i+n_lookback+n_forecast,4]] #esto cambiar por nombre de columna porque si no va a ser horrible 
        #recordar cuál columna es cada índice
        y.append(np.array(label).T)
        # print(np.shape(y))
    print(y[-1])
    return np.array(X),np.array(y)


def fixer_pred(fecha,dataframe,modelo,scaler,columna=0,n_columnas=n_columnas,horas_prediccion=n_forecast):
    """Plotea la predicción de dos días posteriores a la fecha indicada (en formato DD-MM-AAAA HH:00)
    tomando una semana de datos anteriores con el modelo indicado"""
    """ df_input tiene que tener un índice en formato fecha con frecuencia de hora """

    c=dataframe.loc[:fecha]
    c=c[-n_lookback:]
    c_array=c.to_numpy()
    c_array=scaler.transform(c_array)
    c_array=np.reshape(c_array,(1,*c_array.shape))
    print(c_array.shape)
    
    pred=modelo.predict(c_array).flatten()            #datos escalados minmax(0,1)
    
    pred_input_scaler=np.column_stack([np.zeros(shape=(n_forecast,columna)), pred, np.zeros(shape=(n_forecast,n_columnas-1-columna))])
    print(pred_input_scaler.shape)
    
    pred_scaled=scaler.inverse_transform(pred_input_scaler)[:,columna]

    tiempo_forecast=pd.date_range(start=fecha, periods=n_forecast, freq='h')
    tiempo_forecast+=pd.to_timedelta(1, unit='h') #añado una hora porque arranca a predecir después de la fecha indicada
    
    return pd.DataFrame({columnas_a_usar[columna]:pred_scaled},index=tiempo_forecast)[:horas_prediccion]


def plot_pred(fecha,dataframe,modelo,scaler,columna=0,n_columnas=n_columnas,horas_prediccion=n_forecast):
    """Plotea la predicción de dos días posteriores a la fecha indicada (en formato DD-MM-AAAA HH:00)
    tomando una semana de datos anteriores con el modelo indicado"""
    """ df_input tiene que tener un índice en formato fecha con frecuencia de hora """
    fecha_timestamp=pd.Timestamp(fecha)
    añomes=f'{fecha_timestamp.year}-{fecha_timestamp.month}'
    
    feature=columnas_a_usar[columna]
    
    c=dataframe.loc[:fecha]
    c=c[-n_lookback:]
    c_array=c.to_numpy()
    c_array=scaler.transform(c_array)
    c_array=np.reshape(c_array,(1,*c_array.shape))
    print(c_array.shape)
    
    pred=modelo.predict(c_array).flatten()            #datos escalados minmax(0,1)
    
    pred_input_scaler=np.column_stack([np.zeros(shape=(n_forecast,columna)), pred, np.zeros(shape=(n_forecast,n_columnas-1-columna))])
    print(pred_input_scaler.shape)
    
    pred_scaled=scaler.inverse_transform(pred_input_scaler)[:,columna]
 
    tiempo_lookback=c.index
    tiempo_forecast=pd.date_range(start=fecha, periods=n_forecast, freq='h')
    
    tiempo_forecast+=pd.to_timedelta(1, unit='h') #añado una hora porque arranca a predecir después de la fecha indicada
    
    # print(tiempo_lookback[-1])
    # print(tiempo_forecast)
    
    pic1=go.Scatter(x=tiempo_forecast[:horas_prediccion],y=pred_scaled[:horas_prediccion],name='pred')
    pic2=go.Scatter(x=tiempo_lookback, y= c[feature],name='input')
    pic3=go.Scatter(x=dataframe.loc[añomes][feature].index, y=dataframe.loc[añomes][feature],name='x_test_real')
    fig = make_subplots(specs=[[{"secondary_y": False}]])
    fig.add_trace(pic1)
    fig.add_trace(pic2)
    fig.add_trace(pic3)

    fig['layout'].update(title = f'{feature}, predicción y real',xaxis=dict(tickangle=-90))
    # fig['layout'].update(height = 800, width = 1400, title = f'{feature}, predicción y real',xaxis=dict(tickangle=-90))
    iplot(fig)
    # return pd.DataFrame({columnas_a_usar[columna]:pred_scaled},index=tiempo_forecast)


In [9]:
a=df_input.copy().loc['2019-05-01 00:00:00':]
xd=a.diff()
xd=xd[['y', 'wx', 'wy', 'humext', 'pr']]
# px.line(xd['day_cos'])
xd=xd[1:]
aaaaaaaa=xd.mean(axis=1)
filas_con_cero = aaaaaaaa[(aaaaaaaa == 0)]
filas_con_cero
# Calcular la diferencia temporal entre índices consecutivos
time_diff = filas_con_cero.index.to_series().diff()

# Identificar los índices donde el salto es mayor a 1 hora
jump_indices = filas_con_cero.index[time_diff > pd.Timedelta(hours=1)]

print("Índices con saltos mayores a 3 hora:")
print(jump_indices)
time_diff = filas_con_cero.index.to_series().diff()
# time_diff>pd.Timedelta(hours=100)

Índices con saltos mayores a 3 hora:
DatetimeIndex(['2019-06-02 02:00:00', '2019-07-26 16:00:00',
               '2019-07-29 20:00:00', '2019-09-28 10:00:00',
               '2019-09-28 14:00:00', '2019-10-04 04:00:00',
               '2019-10-06 03:00:00', '2021-06-28 22:00:00',
               '2021-06-29 01:00:00', '2021-07-20 20:00:00',
               '2021-08-18 17:00:00', '2021-10-20 01:00:00',
               '2021-11-05 18:00:00', '2021-12-02 02:00:00',
               '2021-12-12 02:00:00', '2022-01-22 03:00:00',
               '2022-01-22 11:00:00', '2022-07-25 01:00:00',
               '2022-07-31 03:00:00', '2022-11-06 05:00:00',
               '2022-11-25 15:00:00', '2022-11-26 06:00:00',
               '2023-12-15 14:00:00', '2024-04-06 22:00:00',
               '2024-07-24 22:00:00', '2024-10-29 14:00:00',
               '2024-11-08 15:00:00'],
              dtype='datetime64[ns]', name='ds', freq=None)


In [10]:
modelos={}
for i,j in enumerate(columnas_a_usar[:5]):
    modelos[f'model{i}']=tf.keras.models.load_model(f'modelos/GRU80_20_{j}_06-12-2024.keras')

df_clon=df.copy()

In [11]:
# #Índices con saltos mayores a 3 hora:
# k=pd.DatetimeIndex(['2019-07-26 16:00:00',
#                '2019-07-29 20:00:00', '2019-09-28 10:00:00',
#                '2019-09-28 14:00:00', '2019-10-04 04:00:00',
#                '2019-10-06 03:00:00', '2021-06-28 22:00:00',
#                '2021-06-29 01:00:00', '2021-07-20 20:00:00',
#                '2021-08-18 17:00:00', '2021-10-20 01:00:00',
#                '2021-11-05 18:00:00', '2021-12-02 02:00:00',
#                '2021-12-12 02:00:00', '2022-01-22 03:00:00',
#                '2022-01-22 11:00:00', '2022-07-25 01:00:00',
#                '2022-07-31 03:00:00', '2022-11-06 05:00:00',
#                '2022-11-25 15:00:00', '2022-11-26 06:00:00',
#                '2023-12-15 14:00:00', '2024-04-06 22:00:00',
#                '2024-07-24 22:00:00', '2024-10-29 14:00:00',
#                '2024-11-08 15:00:00'],
#               dtype='datetime64[ns]', name='ds', freq=None)
# k

In [12]:
k=pd.DatetimeIndex(['2019-07-26 16:00:00',  '2021-06-28 22:00:00',
               '2021-07-20 20:00:00',
               '2021-08-18 16:00:00', 
               '2021-11-05 18:00:00', '2021-12-02 02:00:00',
               '2021-12-12 02:00:00', '2022-01-22 03:00:00',
               '2022-07-25 01:00:00',
               '2022-11-06 04:00:00',
               
               '2023-12-15 14:00:00',
               '2024-10-29 14:00:00',
               ],
              dtype='datetime64[ns]', name='ds', freq=None)
k-=pd.Timedelta(hours=1)
k.shape

(12,)

In [13]:
horas_predecir=[24,6,17,41,48, 7,48 , 48 , 41, 3, 48,46]
indice=0
print(k[indice],horas_predecir[indice])
px.line(df.loc[f'{k[indice].year}-{k[indice].month}']['y'])


2019-07-26 15:00:00 24


In [14]:

# plot_pred(,modelo=model,dataframe=df.loc['2019-05':][columnas_a_usar],scaler=np80_scaler)
for i in range(5):
    plot_pred(k[indice],modelo=modelos[f'model{i}'],dataframe=df.loc['2019-05':][columnas_a_usar],scaler=np80_scaler,columna=i,horas_prediccion=42) 

(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step
(48, 9)


(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step
(48, 9)


(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step
(48, 9)


(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
(48, 9)


(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
(48, 9)


In [15]:
# horas_predecir=[24,6,17,40,48, 48,7,48 , 48 , 48 , 41, 3, 48,46]
# len(horas_predecir)

In [16]:
horas_predecir=[24,6,17,21,48, 7,48 , 48 , 41, 3, 48,46]

df_clon=df.copy()

for i in range(5):
    for p,j in enumerate(horas_predecir):
        temp=fixer_pred(k[p],modelo=modelos[f'model{i}'],dataframe=df.loc['2019-05':][columnas_a_usar],scaler=np80_scaler,columna=i,horas_prediccion=j)
        # print(temp)
        df_clon.update( temp )

(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(48, 9)
(1, 168, 9)
1/

In [28]:
indice=8
print(k[indice],horas_predecir[indice])
# px.line(df_clon.loc[f'{k[indice].year}-{k[indice].month}']['y'])

pic1=go.Scatter(x=df.loc[f'{k[indice].year}-{k[indice].month}'].index, y=df.loc[f'{k[indice].year}-{k[indice].month}']['y'],name='huecos')
pic2=go.Scatter(x=df_clon.loc[f'{k[indice].year}-{k[indice].month}'].index, y=df_clon.loc[f'{k[indice].year}-{k[indice].month}']['y'],name='modificado')

fig = make_subplots(specs=[[{"secondary_y": False}]])
fig.add_trace(pic1)
fig.add_trace(pic2)
fig['layout'].update( title = 'temperatura, predicción y real',xaxis=dict(tickangle=-90))
iplot(fig)


2022-07-25 00:00:00 41


In [18]:
k2=pd.DatetimeIndex(['2021-11-07 17:00:00','2021-12-14 01:00:00',
                     '2022-01-24 02:00:00'

                     ])

horas_predecir2=[40,8,48]

k2

DatetimeIndex(['2021-11-07 17:00:00', '2021-12-14 01:00:00',
               '2022-01-24 02:00:00'],
              dtype='datetime64[ns]', freq=None)

In [19]:
plot_pred(k2[0],modelo=modelos['model0'],dataframe=df_clon.loc['2019-05':][columnas_a_usar],scaler=np80_scaler,columna=0,horas_prediccion=40)

(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
(48, 9)


In [20]:
df_clon2=df_clon.copy()
for i in range(5):
    for p,j in enumerate(horas_predecir2):
        temp=fixer_pred(k2[p],modelo=modelos[f'model{i}'],dataframe=df_clon.loc['2019-05':][columnas_a_usar],scaler=np80_scaler,columna=i,horas_prediccion=j)
        # print(temp)
        df_clon2.update( temp )

(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)


In [21]:
indice=2
print(k2[indice],horas_predecir2[indice])
# px.line(df_clon.loc[f'{k[indice].year}-{k[indice].month}']['y'])

pic1=go.Scatter(x=df_clon.loc[f'{k2[indice].year}-{k2[indice].month}'].index, y=df_clon.loc[f'{k2[indice].year}-{k2[indice].month}']['y'],name='huecos')
pic2=go.Scatter(x=df_clon2.loc[f'{k2[indice].year}-{k2[indice].month}'].index, y=df_clon2.loc[f'{k2[indice].year}-{k2[indice].month}']['y'],name='modificado')

fig = make_subplots(specs=[[{"secondary_y": False}]])
fig.add_trace(pic1)
fig.add_trace(pic2)
fig['layout'].update( title = 'temperatura, predicción y real',xaxis=dict(tickangle=-90))
iplot(fig)

2022-01-24 02:00:00 48


In [22]:
k2=pd.DatetimeIndex(['2022-01-26 02:00:00','2022-01-28 02:00:00',
                     '2022-01-30 02:00:00'
                     ])

horas_predecir2=[48,48,46]

k2

for i in range(5):
    for p,j in enumerate(horas_predecir2):
        temp=fixer_pred(k2[p],modelo=modelos[f'model{i}'],dataframe=df_clon2.loc['2019-05':][columnas_a_usar],scaler=np80_scaler,columna=i,horas_prediccion=j)
        # print(temp)
        df_clon2.update( temp )

(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
(48, 9)
(1, 168, 9)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
(48, 9)


In [23]:
indice=1
print(k2[indice],horas_predecir2[indice])
# px.line(df_clon.loc[f'{k[indice].year}-{k[indice].month}']['y'])

pic1=go.Scatter(x=df_clon.loc[f'{k2[indice].year}-{k2[indice].month}'].index, y=df_clon.loc[f'{k2[indice].year}-{k2[indice].month}']['y'],name='huecos')
pic2=go.Scatter(x=df_clon2.loc[f'{k2[indice].year}-{k2[indice].month}'].index, y=df_clon2.loc[f'{k2[indice].year}-{k2[indice].month}']['y'],name='modificado')

fig = make_subplots(specs=[[{"secondary_y": False}]])
fig.add_trace(pic1)
fig.add_trace(pic2)
fig['layout'].update( title = 'temperatura, predicción y real',xaxis=dict(tickangle=-90))
iplot(fig)

2022-01-28 02:00:00 48


In [29]:
df_clon2.columns

Index(['y', 'humext', 'pr', 'rafaga 10m', 'tasa precip', 'lluvia hoy', 'pres',
       'cont lluvia', 'tempint', 'humint', 'viento actual', 'wind chill',
       'heat index', 'tempap', 'radsolar', 'dir actual', 'lluvia desde 00:00',
       'wx', 'wy', 'ds', 'day_sin', 'day_cos', 'year_sin', 'year_cos'],
      dtype='object')

In [30]:
df_clon2.to_parquet('datos15til24resampledwxwy_sinhuecos.parquet.gzip',compression='gzip')